# Creating of a Flask application showing students' academic performance

In this segment we will create a gradebook where we will have an information about the students, labs and what they got for these labs. All the information we will store in a SQLite database and then we will display it on the webpage. It will look like:

<img src="images/gradebook.jpg">

First of all, let's create a new folder `"gradebook"`, where will store all the files for this project and also `"app.py"` file, folder `"templates"` with `"index.html"` file and stop the earlier run Flask server (you may do this by selection of **Kernel ➜ Interrupt** on the command cell where **`%run app.py`** was executed). 

And now let's create a database "students" with two tables "students" and "points" with such fields and relations between tables (it is a simple example of one-to-many relationship; let's note, that such tables could be presented as many-to-many relationship where labs might be separated into a single table; but we stopped on the selected type of relationship because it is the most often used):

<img src="images/sql.jpg">

and fill it with data (numbers in brackets near `VARCHAR` denotes the maximal number of characters in a field value; the joining line shows between which fields the relationship exists):

In [1]:
import sqlite3  # This is the Python module for working with SQLite 

conn = sqlite3.connect("./gradebook/students.db")  # In this way we are joining with the database in "gradebook" folder
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name VARCHAR(50),
        last_name VARCHAR(50)
    );
""")  
cursor.execute("""
    CREATE TABLE IF NOT EXISTS points (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        student_id INTEGER,
        lab VARCHAR(100),
        points INTEGER
    ); 
""");
conn.commit()

We are trying to connect to SQLite database with the name `"students.db"` in the `"gradebook"` folder. As there is no database with this name in this folder, Python will create a new one and connect to it. Once you have a `Connection`, you can create a `Cursor` object and call its `execute()` method to perform SQL commands and finally to save results using `commit()` method. 

Thus, we have such tables:

In [2]:
list(cursor.execute('SELECT name FROM sqlite_master WHERE type = "table";'))

[('students',), ('sqlite_sequence',), ('points',)]

Now let's fill them with some data

In [3]:
# We will add points between 1 and 5 in a random way
from random import randint

# Define few students
students = [
    {"first_name": "George", "last_name": "Clooney"},
    {"first_name": "Sandra", "last_name": "Bullock"},
    {"first_name": "Matt", "last_name": "Damon"},
]
# and few labs
labs = ["Lab 1", "Lab 2", "Lab 3", "Lab 4", "Lab 5"]
    
# Insert data into the table "students"
for i in students:
    cursor.execute("INSERT INTO students (first_name, last_name) VALUES (?, ?);", (i["first_name"], i["last_name"]))
# Insert data into the table "points"
for i in range(len(students)):
    for lab in labs:
        cursor.execute("INSERT INTO points (student_id, lab, points) VALUES (?, ?, ?);", (i+1, lab, randint(1,5)))
        # We have added one to `i` because Python starts indexing from 0 when SQLite primary keys begins from 1
conn.commit()

Let's see the `"students"` records

In [4]:
for i in cursor.execute("SELECT * FROM students;"):
    print(i)
conn.close()

(1, 'George', 'Clooney')
(2, 'Sandra', 'Bullock')
(3, 'Matt', 'Damon')


Currently, we are going to display these data in HTML page in a table format. An HTML table is defined with the [`<table>` tag](https://www.w3schools.com/html/html_tables.asp). Each table row is defined with the `<tr>` tag. A table header is defined with the `<th>` tag. By default, table headings are bold and centered. A table data/cell is defined with the `<td>` tag. 

Let's transfer the above code for records retrieving to the `"gradebook/app.py"` file

```python
from flask import Flask, render_template
import sqlite3

app = Flask(__name__)
    
@app.route('/')
def home():
    conn = sqlite3.connect("./gradebook/students.db") 
    cursor = conn.cursor()
    # Retrieve records of all tables and construct dictionaries for convenient usage at HTML tamplates
    points = cursor.execute("SELECT * FROM points;")
    points = [{"id": i[0], "student_id": i[1], "lab": i[2], "points": i[3]} for i in points]
    students = [{"id": i[0], "first_name": i[1], "last_name": i[2]} for i in cursor.execute("SELECT * FROM students;")]
    labs = [{"title": i[0]} for i in cursor.execute("SELECT DISTINCT(lab) FROM points;")]
    return render_template('index.html', students=students, labs=labs, points=points)

if __name__ == '__main__':
    app.run(debug=True)
```

and add the following code to the `"gradebood/templates/index.html"` file

```html
<!DOCTYPE html>
<html>
<body>   <!-- This is a comment in HTML -->
    <table>
        <thead>
            <tr>    <!-- Construct table header -->
                <th>Student</th>
                {% for lab in labs %}
                    <th>{{ lab.title }}</th>
                {% endfor %}
                <th>Total</th>
            </tr>
        </thead>
        <tbody>
            {% for student in students %}    <!-- Fill table rows for each student --> 
                <tr>
                    <td>{{ student.first_name }} {{ student.last_name }}</td>
            <!-- The `set` tag allows assigning values to variables. We will use `total` to count total points -->
                    {% set total = [0] %}     
                    {% for lab in labs %}    <!-- Find respective lab's number -->
            <!-- This variable will be used to check whether the student performed the lab and got some points -->
                        {% set value = {"point": 0} %}    
                        {% for point in points %}
                            {% if point.student_id == student.id and point.lab == lab.title %}  
                                <!-- Update the value of `value` variable with the current points -->
                                {% set _ = value.update({"point": point.points}) %}
                            {% endif %}
                        {% endfor %}
                        <td>
                            {% if value["point"] > 0 %}    <!-- If the student got points for the lab -->
                                {{ value["point"] }}
                                <!-- Increase value of total points -->
                                {% set _ = total.append(total.pop()+ value["point"]) %}    
                            <!-- If the student didn't perform the lab, we fill the corresponding cell with a dash -->
                            {% else %}    
                                -
                            {% endif %}
                        </td>
                    {% endfor %}
                    <td>{{ total[0] }}</td>
                </tr>
            {% endfor %}
         </tbody>
    </table>
</body>
</html>
```

To build HTML structure convenient fot displaying disariable table we have used Jinja2 syntax, particularly, `{% set %}` tag which replies for assignment. More info you can find [here](http://jinja.pocoo.org/docs/2.9/templates/). 
<br/>
In Jinja2, `{% set %}` tag doesn't assign a variable to global context. That's why we create a list object for `total` variable and a dictionary for `value` (both to show variations), because a reassignment of an integer will change its value inside a loop, but not outside of it.

Run this code to see how the table looks

In [ ]:
%run gradebook/app.py

# Flask ORM & migrations

We will use the [Flask-SQLAlchemy](http://flask-sqlalchemy.pocoo.org/2.1/) extension to manage our application, because the usage of direct queries is really not convenient. This extension provides a wrapper for the SQLAlchemy project, which is an [Object Relational Mapper (ORM)](https://en.wikipedia.org/wiki/Object-relational_mapping). ORMs allow database applications to work with objects instead of tables and SQL. The operations performed on the objects are translated into database commands transparently by the ORM. 

First of all, let's install Flask-SQLAlchemy

In [ ]:
!pip install Flask-SQLAlchemy

Then import Flask-SQLAlchemy library in `"app.py"`. We will not use SQLite in the flask application anymore, so replace line `import sqlite3` with this one:

```python
from flask_sqlalchemy import SQLAlchemy
```

Then right after instance `app` of the `Flask` class creation add config variable with using SQLite database like below

```python
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///students.db'
```

and finally in the next line create the SQLAlchemy object (we call it `db`) by passing it the application 

```python
db = SQLAlchemy(app)
```

Once created, that object then contains all the functions and helpers from both `sqlalchemy` and `sqlalchemy.orm`. Furthermore it provides a class called `Model` that is a declarative base which can be used to declare models. Let's create the model `Student` corresponding to the described above `"student_points"` table. To do this, create a new file `"models.py"` nearby `"app.py"` with the following content

```python
from app import db

class Student(db.Model):
    __tablename__ = 'students'
    __table_args__ = {'extend_existing': True}

    id = db.Column(db.Integer, primary_key=True)
    first_name = db.Column(db.String(50))
    last_name = db.Column(db.String(50))
    points = db.relationship('Point', backref='students', lazy='dynamic')


class Point(db.Model):
    __tablename__ = 'points'
    __table_args__ = {'extend_existing': True}

    id = db.Column(db.Integer, primary_key=True)
    lab = db.Column(db.String(100))
    points = db.Column(db.Integer)
    student_id = db.Column(db.Integer, db.ForeignKey('students.id'))
```

Above the attribute `__tablename__` sets the name of table. Currently, we set the same names as previously to migrate all records from `"students"` and `"points"` SQLite tables (we will explain further how to do this). Otherwise thees tables will be empty. The attribute `__table_args__` with key `extend_existing = True` allows to redefine options and columns on an existing Table object. Both these attributes are not required, particularly, if `__tablename__` is not defined, the table will be given the same name as the respective class (pespectively, `"student"` and `"point"` in our case). We use them to migrate already existing data. 
<br/>
Then we create all fields present in `"students"` and `"points"` tables using `Column` class where we must define the field type (all available data types you may see [here](http://flask-sqlalchemy.pocoo.org/2.1/models/)). The name of the column is the name you assign it to. If you want to use a different name in the table you can provide an optional first argument which is a string with the desired column name. 
<br/>
The field `id` in both classes is defined as primary key by using `primary_key=True`. 
<br/>
Relationships are expressed with the `relationship()` function. That function returns a new property that can do multiple things. In this case we told it to point to the `Point` class and load multiple of those. How does it know that this will return more than one address? Because SQLAlchemy guesses a useful default from your declaration set in `backref` attribute. `lazy` defines when SQLAlchemy will load the data from the database. When it is `'dynamic'`, then instead of loading the items SQLAlchemy will return another query object which you can further refine before loading the items. This is usually what you want if you expect more than a handful of items for this relationship.

After `"models.py"` file creation we need to import it in the `"app.py"`. Add 

```python
import models
```

right after line `db = SQLAlchemy(app)` (such positioning is principal, otherwise object will be not importable).

To create the initial database, let's create `"create_db.py"` file in the same folder where `"app.py"` lies and copy and past there the folliwing code that creates the tables and database:

```python
from app import db

db.create_all()
```

After that execute this file

In [ ]:
%run gradebook/create_db.py

Of course, you could execute the above command from Python shell, so the creation of this file is not required.

The above operations create by default empty database and respective table(s) (also empty). But what to do if we already have some data and want you them? Or what do if during web application development we decided to create some additional columns in a table or even new tables with cross-relationships? 

One of much challenging aspects of working with relational databases is making changes to the database schema, such as adding or deleting columns. During the early development of an application, making changes to a table in a relational database is easy if you’re not worried about deleting all of the data in your database. However, once you get into production and are actually storing real data in your relational database, you need to be very cautious when changing a table. While relational databases have a lot of strong points, being able to easily update the underlying schema of a database table is not one of them.

To be explicit, the use of the [Flask-Migrate](https://flask-migrate.readthedocs.io/en/latest/) module is intended for Flask applications that are using SQLAlchemy for interfacing with the underlying database. We are not going to go deeply in the logic of migrations. We should just understand what they can do and what for.

To make Flask-Migrate's command more friendly the [Flask-Script](https://flask-script.readthedocs.io/en/latest/) extension that provides support for writing external scripts in Flask is used.

Let's install both them

In [ ]:
!pip install Flask-Migrate
!pip install Flask-Script

To do migrations, you need to create a new file `"manage.py"` with the following content

```python
from flask_script import Manager
from flask_migrate import Migrate, MigrateCommand
from app import app, db

migrate = Migrate(app, db)
manager = Manager(app)
manager.add_command('db', MigrateCommand)

if __name__ == '__main__':
    manager.run()
```

where `Migrate(app, db)` makes possible doing migrations of data and the next two lines allows runing commands for migrations from command shell.

With the above application you can create a migration repository with the following command:

In [ ]:
%run gradebook/manage.py db init --directory=gradebook/migrations

This will add a `"migrations"` folder to your application (look whether it was created). The contents of this folder need to be added to version control along with your other source files. Note we set `directory` option that points to the directory containing the migration scripts. If this argument is omitted the directory used is migrations.

You can then generate an initial migration:

In [ ]:
%run gradebook/manage.py db migrate --directory=gradebook/migrations

Then each time the database models change repeat the migrate and upgrade 

```bash
$  python manage.py db upgrade
```
commands 

Now that you have declared models it’s time to query the data from the database. SQLAlchemy ORM allows insering, deleting and quering. 

Particularly, to **insert** some data into database we should firstly to create a Python object of respective model class, then add it to the SQLAlchemy session and finally commit the session. For example, creation of a new student can be done as follows (we will use these command below)

```python
>>> student = models.Student(first_name="New", last_name="User")  # Create a new student
>>> point1 = models.Point(lab="Lab 1", points=5)  # Create new point
>>> point2 = models.Point(lab="Lab 2", points=4)  # And the point for another lab
>>> student.points = [point1, point2]  # Add these points to the just created student
>>> db.session.add(student)  # Add the student to the session
>>> db.session.commit()  # Commit changes
```

**Deleting** records is very similar, instead of `add()` use `delete()`:

```python
>>> db.session.delete(student)
>>> db.session.commit()
```

To **get data back** out of our database use a `query` attribute on your Model class. It returns a new query object over all records. With `all()` or `first()` you may return all records or only the first one, respectively. `query` can be applied to both Model and session. 

Let's look how it works and retrieve the same table data as before, but now by using ORM. Please change the `home()` view in the `"app.py"` to this 

```python
def home():
    students = models.Student.query.all()
    points = models.Point.query.all()
    labs = db.session.query(models.Point.lab.distinct().label("title")).all()
    return render_template('index.html', students=students, labs=labs, points=points)
```

and update the http://127.0.0.1:5000/.

Above we retrieve all records from `"student"` and `"points"` tables (first two queries) and get all unique lab titles using `distinct()` method. To provide the same key name (`"title"`) as it was defined earlier. 

## Expand the functional of the web application

Now, we are going to create a new HTML page, where two text fields for typing student's name and surname were displayed and a button to submit creation of a new student will be present.
<br/>
First of all, let's create a new HTML template `"add_new_student.html"` in the `"gradebook/templates"` with the following content

```html
<!DOCTYPE html>
<html>
<body>
    <a href="{{ url_for('home') }}">Go to the gradebook</a>
    <form action="{{ url_for('submit_new_student') }}" method="POST">
        <label for="first_name">First Name</label><br>
        <input type="text" name="first_name" placeholder="Name"/><br>
        <label for="last_name">Last Name</label><br>
        <input type="text" name="last_name" placeholder="Surname"/><br>
        <input type="submit" value="Submit" />
    </form>
</body>
</html>
```

where we have created the link to the web page with gradebook using `<a>` HTML tag (for this aim we used `{{ url_for('home') }}` structure where `url_for` function put the URL defined in `@app.route()` decorator near `home()` function). Then we create a form for collection of user inputs with help of HTML `<form>` element. 
<br/>
The `<input>` element is the most important form element. It can be displayed in several ways, depending on the `type` attribute (particularly, `type="text"` defines a one-line text input field, `type="radio"` defines a radio button (for selecting one of many choices), `type="submit"` defines a submit button (for submitting the form), etc.; more info you can find [here](https://www.w3schools.com/tags/tag_form.asp)). All text written in `<input type="text">` will be processed by flask (see below). Attribute `placeholder` contains text that will be positioned in the text input. `<input type="submit">` sends data to backend using POST method (it is one of HTTP request methods; two commonly used methods for a request-response between a client and server are: GET and POST; GET requests data from a specified resource when POST submits data to be processed to a specified resource; additional information you can find [here](https://www.w3schools.com/tags/ref_httpmethods.asp)) to the function `submit_new_student()` defined in `action` attribute (we will create it further). 
<br/>
The `<label>` tag defines a text label for an `<input>` element. It provides a usability improvement for mouse users, because if the user clicks on the text within the `<label>` element, it toggles the control.

Now let's add the link to the page for new student creation to the `"index.html"`. Above the openning `<table>` tag (just after openning `<body>` tag) add the following code

```html
<a href="{{ url_for('add_new_student') }}">Add new student</a>
```

After that in `"app.py"` create two new functions

```python
@app.route('/new-student')
def add_new_student():
	return render_template('add_new_student.html')

@app.route('/submit-new-student', methods=['POST'])
def submit_new_student():
	first_name = request.form['first_name']
	last_name = request.form['last_name']
	student = models.Student(first_name=first_name, last_name=last_name)
	db.session.add(student)
	db.session.commit()
	return redirect(url_for('add_new_student'))
```

The function `add_new_student()` opens `"add_new_student.html"` template in http://127.0.0.1:5000/new-student. 
<br/>
The function `submit_new_student()` gets data from the form (to get the value of an input we use `request.form['input_name']` command, where `input_name` corresponds the value of `name` attribute of `<input>` tag), creates and save a new student with name and surname defined by the user in the inputs and finally updates the page http://127.0.0.1:5000/new-student with the help of `redirect()` method that redirects a user to another endpoint.

Also we need import `request`, `redirect` and `url_for` from `flask`.

The next gif animation demonstrates how it looks now

<img src="images/add_new_student.gif">

Thus, we have possibility to create new users, but they have no points for any lab. Let's create functionality of adding new points for users.

At first, add an additional link to the `"index.html"` just after `<a href="{{ url_for('add_new_student') }}">Add new student</a>` that will open the page for adding points for selected student and lab pair (the list of existing students and labs will be displayed as a dropdown where we will be able to select an item; such dropdown in HTML structure is represented by the [`<select>` tag](https://www.w3schools.com/tags/tag_select.asp); the `<option>` tags inside the `<select>` element define the available options in the list; see below):

```html
<a href="{{ url_for('add_new_student') }}">Add new student</a> <a href="{{ url_for('add_new_grade') }}">Add new grade</a>
```

Create a new HTML template file `"add_new_grade.html"` in the `"gradebook/templates"` folder and past there the below code

```html
<!DOCTYPE html>
<html>
<body>
    <a href="{{ url_for('home') }}">Go to the gradebook</a>
    <form action = "{{ url_for('submit_new_grade') }}" method = "POST">
    	<p>Select a student</p>
    	<select name="student_id">
			{% for student in students %}
	        	<option value="{{ student.id }}">{{ student.first_name }} {{ student.last_name }}</option>
	        {% endfor %}
    	</select>
    	<p>Select a lab</p>
    	<select name="lab_title">
    		<option value="None">I want to add a new title</option>
			{% for lab in labs %}
	        	<option value="{{ lab.title }}">{{ lab.title }}</option>
	        {% endfor %}
    	</select><br>
    	<label for="new_lab">or create a new lab</label><br>
        <input type="text" name="new_lab" placeholder="Title"/><br>
        <label for="grade">Points</label><br>
        <input type="number" name="grade" min="1" max="5"><br>
        <input type="submit" value="Submit"/>
    </form>
</body>
</html>
```

where we at first propose to an user to select an existing lab's title from the dropdown `<select name="lab_title">` and then to type a new one in `<input type="text" name="new_lab" placeholder="Title"/>` if he wants to create a new lab. To write only integer numbers in an input field the `<input>` with `type="number"` is used. We suppose, that points can lie between 1 and 5; attributes `min` and `max` allow setting ranges of available numbers.

Then add the following two functions to the`"app.py"`

```python
@app.route('/new-grade')
def add_new_grade():
	students = models.Student.query.all()   # Retrieve all students, because we show them as select's options
	labs = db.session.query(models.Point.lab.distinct().label("title")).all()  # We also show all labs
	return render_template('add_new_grade.html', students=students, labs=labs)

@app.route('/submit-new-grade', methods=['POST'])
def submit_new_grade():
	student_id = int(request.form['student_id'])  # We must know for which student the user wants to add poins 
	lab_title = request.form['lab_title']  # Here we select the lab's title from the list of existing labs
	if lab_title == "None":   # If no existing lab's title was selected
		lab_title = request.form['new_lab']  # then we read the typed lab title
	# We should convert the entered points from string type to integer one because points has such type in `Point` model
    points = int(request.form['grade'])  
	point = models.Point(lab=lab_title, points=points, student_id=student_id) # Create a new record to the "points" table
	db.session.add(point)
	db.session.commit()
	return redirect(url_for('add_new_grade'))  # Update page
```

The result will look like this:

<img src="images/add_new_grade.gif">

> # Exercise 2

> Add a new link "Delete student" to the page with table that will open a new web page `"/delete-student"`. This page should contain link to the main page (with the table), dropdown (`<select>` HTML tag) with all existing students where we can select the student we are going to delete from the table and the button "Delete". 

> **Hint:** To delete a record from the table, you may use the `db.session.delete()` function described above.

> # Exercise 3

> Add a new link "Edit grade" that will open a new web page `"/update-grade"` where a user can update a grade for the selected student and lab pair. This page should contain a link to the main page, a dropdown with students, a dropdown with lab titles, a text input where a user can type a new grade and the button "Submit".

> **Hint:** To update, for example, the name of the student with `id = 1` (it is George Clooney) we can use such approach
> ```python
clooney = models.Student.query.get(1)  # Get the user with id = 1
clooney.first_name = "Adam"  # Change his name
db.session.commit()  # Save changes
```

> # Exercise 4

> Add a link "Order by total" that will update the table (the URL should not be changed) and order records in it by the total points in descending order, i.e. the best student should be on the top.

Honestly, the build HTML pages are not beautiful. Moreover, such way of web development usually brings a lot of problems when large and complex HTML pages are creating. [Twitter's Bootstrap](http://getbootstrap.com/) is an excellent set of carefully crafted user interface elements, layouts and JavaScript tools, easily available to use in your web design project. All you need is to create a link to the [Bootstrap resources](http://getbootstrap.com/getting-started/#download-cdn) in the HTML head and use the predefined CSS styles.

We are not going to teach Bootstrap in this lesson. We will simply show an example of the main page customization with the help of Bootstrap classes (see below). You are recommended to familiarize yourself with the basic possibilities of Bootstrap using the official resources ([this](http://getbootstrap.com/css/) and [this](http://getbootstrap.com/components/)).
        
```html
<!DOCTYPE html>
<html>
<head>
	<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
</head>
<body>
	<div class="container">
		<div class="panel panel-info">
			<div class="panel-heading">
				<h1 class="panel-title">Students gradebook</h1>
			</div>
			<div class="panel-body">
				<div class="row">
					<div class="col-md-offset-1 col-md-1">
			    		<a class="btn btn-default" href="{{ url_for('add_new_student') }}">Add new student</a> 
			    	</div>
			    	<div class="col-md-offset-1 col-md-1">
			    		<a class="btn btn-default" href="{{ url_for('add_new_grade') }}">Add new grade</a> 
			    	</div>
			    	<div class="col-md-offset-1 col-md-1">
                        <a class="btn btn-danger" href="{{ url_for('delete_student') }}">Delete student</a> 
                    </div>
                    <div class="col-md-offset-1 col-md-1">
                        <a class="btn btn-info" href="{{ url_for('edit_grade') }}">Edit grade</a> 
                    </div>
                    <div class="col-md-offset-1 col-md-1">
                        <a class="btn btn-sucsess" href="{{ url_for('sort_students') }}">Order by total</a> 
                    </div>
			    </div>
			    <hr/>
	    		<div class="row col-md-offset-1 col-md-10 col-md-offset-1">
				    <table class="table table-striped table-bordered table-hover">
				        <thead>
				            <tr class="success">
				                <th>Student</th>
				                {% for lab in labs %}
				                    <th>{{ lab.title }}</th>
				                {% endfor %}
				                <th>Total</th>
				            </tr>
				        </thead>
				        <tbody>
				            {% for student in students %}
				                <tr>
				                    <td>{{ student.first_name }} {{ student.last_name }}</td>
				                    {% set total = [0] %}
				                    {% for lab in labs %}
				                        {% set value = {"point": 0} %}
				                        {% for point in points %}
				                            {% if point.student_id == student.id and point.lab == lab.title %}  
				                                {% set _ = value.update({"point": point.points}) %}
				                            {% endif %}
				                        {% endfor %}
				                        <td>
				                            {% if value["point"] > 0 %}
				                                {{ value["point"] }}
				                                {% set _ = total.append(total.pop()+ value["point"]) %}
				                            {% else %}
				                                -
				                            {% endif %}
				                        </td>
				                    {% endfor %}
				                    <td class="warning"><strong>{{ total[0] }}</strong></td>
				                </tr>
				            {% endfor %}
				         </tbody>
				    </table>
				</div>
			</div>
        </div>
	</div>
</body>
</html>
```

> # Exercise 5

> Learn Bootstrap and add some components and classes to pretify all other pages to yor taste.